In [1]:
import datetime
import ast
import json

import numpy as np
import pandas as pd
import string

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor

#### Read Dataset

In [2]:
df = pd.read_csv('cars.csv')
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,transmission,engine,vin,stock_number,vehicle_history,seller_name,price_history
0,https://www.cars.com/vehicledetail/5ee3ae04-69...,2018 Hyundai Santa Fe Sport 2.4L,"79,497 mi.","$17,899",Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,6-Speed Automatic,2.4L I4 16V GDI DOHC,5XYZT3LB3JG542718,13669,"{'Accidents or damage': 'None reported', '1-ow...",ETHIO Motors,"[('9/21/21', 'Listed', '$20,299'), ('2/04/22',..."
1,https://www.cars.com/vehicledetail/c99e5035-76...,2017 RAM 1500 Express,"80,326 mi.","$27,900",This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,3C6RR7KT1HG533082,KBB2955,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('3/15/22', 'Listed', '$30,985'), ('6/01/22',..."
2,https://www.cars.com/vehicledetail/4e79fa3a-ba...,2019 Nissan Kicks SR,"92,959 mi.","$18,950",This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,Automatic CVT,1.6L I4 16V MPFI DOHC,3N1CP5CU6KL510673,KBB3225,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('4/16/22', 'Listed', '$19,980'), ('6/01/22',..."
3,https://www.cars.com/vehicledetail/86a0d096-32...,2019 RAM 1500 Laramie,"59,176 mi.","$39,950",Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,1C6SRFDT5KN869809,KBB3246,{'Accidents or damage': 'At least 1 accident o...,Custom Cars West,"[('6/01/22', 'Listed', '$40,523'), ('7/14/22',..."
4,https://www.cars.com/vehicledetail/36efbef5-f2...,2015 Jeep Wrangler Unlimited Rubicon,"135,385 mi.","$26,950",This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,-,3.6L V6 24V MPFI DOHC,1C4BJWFG2FL530006,KBB3145,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('1/21/22', 'Listed', '$29,950'), ('3/22/22',..."


## Split vehicle_history to 4 parts 
#### Need to adjust this, to more properly deal with dictionaries

In [3]:
# original code, can remove when confirm that it was properly dealt with

#df[['accident','one_owner','personal_use','open_recall']] = df.vehicle_history.str.split(',',3, expand=True)
#df.head()

def extract_dict(s):
    try:
        # Convert the string to a dictionary using literal_eval
        return ast.literal_eval(s)
    except (SyntaxError, ValueError):
        # Handle cases where literal_eval fails, e.g., malformed strings
        return {}

# First convert dtype to str from object
df['vehicle_history'] = df['vehicle_history'].astype(str)
# Apply the extract_dict fcn
df['vehicle_history'] = df['vehicle_history'].apply(extract_dict)
# normalize and join
norm = pd.json_normalize(df['vehicle_history'])
df = pd.concat([df, norm], axis= 1)
df

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,...,vin,stock_number,vehicle_history,seller_name,price_history,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title
0,https://www.cars.com/vehicledetail/5ee3ae04-69...,2018 Hyundai Santa Fe Sport 2.4L,"79,497 mi.","$17,899",Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,...,5XYZT3LB3JG542718,13669,"{'Accidents or damage': 'None reported', '1-ow...",ETHIO Motors,"[('9/21/21', 'Listed', '$20,299'), ('2/04/22',...",None reported,Yes,No,At least 1 open recall reported,NaN
1,https://www.cars.com/vehicledetail/c99e5035-76...,2017 RAM 1500 Express,"80,326 mi.","$27,900",This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,...,3C6RR7KT1HG533082,KBB2955,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('3/15/22', 'Listed', '$30,985'), ('6/01/22',...",None reported,No,Yes,NaN,NaN
2,https://www.cars.com/vehicledetail/4e79fa3a-ba...,2019 Nissan Kicks SR,"92,959 mi.","$18,950",This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,...,3N1CP5CU6KL510673,KBB3225,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('4/16/22', 'Listed', '$19,980'), ('6/01/22',...",None reported,No,Yes,NaN,NaN
3,https://www.cars.com/vehicledetail/86a0d096-32...,2019 RAM 1500 Laramie,"59,176 mi.","$39,950",Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,...,1C6SRFDT5KN869809,KBB3246,{'Accidents or damage': 'At least 1 accident o...,Custom Cars West,"[('6/01/22', 'Listed', '$40,523'), ('7/14/22',...",At least 1 accident or damage reported,No,No,At least 1 open recall reported,NaN
4,https://www.cars.com/vehicledetail/36efbef5-f2...,2015 Jeep Wrangler Unlimited Rubicon,"135,385 mi.","$26,950",This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,...,1C4BJWFG2FL530006,KBB3145,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('1/21/22', 'Listed', '$29,950'), ('3/22/22',...",None reported,No,Yes,At least 1 open recall reported,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,https://www.cars.com/vehicledetail/58470668-1c...,2022 Toyota Tacoma TRD Off Road,"32,443 mi.","$36,749","Great Deal $2,494 under",-,-,Four-wheel Drive,18-22,Gasoline,...,3TMCZ5AN6NM488353,55846,"{'Accidents or damage': 'None reported', 'Clea...",Hertz Car Sales Denver,[],None reported,Yes,No,NaN,Yes
5998,https://www.cars.com/vehicledetail/0d3c6ec5-e0...,2022 Nissan Kicks SV,"35,000 mi.","$18,834",Good Deal $74 under,Fresh Powder,Charcoal,Front-wheel Drive,31-36,Gasoline,...,3N1CP5CV8NL524068,55861,"{'Accidents or damage': 'None reported', 'Clea...",Hertz Car Sales Denver,"[('10/28/23', 'Listed', '$19,324'), ('11/01/23...",None reported,Yes,No,NaN,Yes
5999,https://www.cars.com/vehicledetail/7b40bf89-08...,2022 Ford Expedition XLT,"23,477 mi.","$50,599",This is a good deal.,Oxford White,Black,Four-wheel Drive,16-22,Gasoline,...,1FMJU1JT3NEA58455,55835,"{'Accidents or damage': 'None reported', 'Clea...",Hertz Car Sales Denver,[],None reported,Yes,No,NaN,Yes
6000,https://www.cars.com/vehicledetail/0fc31017-d3...,2022 Nissan Altima 2.5 S,"47,518 mi.","$18,199",Good Deal $271 under,Brilliant Silver Metallic,Charcoal,Front-wheel Drive,28-39,Gasoline,...,1N4BL4BV8NN330081,55833,"{'Accidents or damage': 'None reported', 'Clea...",Hertz Car Sales Denver,[],None reported,Yes,No,NaN,Yes


In [4]:
df[['year','make','model']] = df.listing_title.str.split(' ',2, expand=True)
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,...,seller_name,price_history,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,year,make,model
0,https://www.cars.com/vehicledetail/5ee3ae04-69...,2018 Hyundai Santa Fe Sport 2.4L,"79,497 mi.","$17,899",Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,...,ETHIO Motors,"[('9/21/21', 'Listed', '$20,299'), ('2/04/22',...",None reported,Yes,No,At least 1 open recall reported,NaN,2018,Hyundai,Santa Fe Sport 2.4L
1,https://www.cars.com/vehicledetail/c99e5035-76...,2017 RAM 1500 Express,"80,326 mi.","$27,900",This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,...,Custom Cars West,"[('3/15/22', 'Listed', '$30,985'), ('6/01/22',...",None reported,No,Yes,NaN,NaN,2017,RAM,1500 Express
2,https://www.cars.com/vehicledetail/4e79fa3a-ba...,2019 Nissan Kicks SR,"92,959 mi.","$18,950",This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,...,Custom Cars West,"[('4/16/22', 'Listed', '$19,980'), ('6/01/22',...",None reported,No,Yes,NaN,NaN,2019,Nissan,Kicks SR
3,https://www.cars.com/vehicledetail/86a0d096-32...,2019 RAM 1500 Laramie,"59,176 mi.","$39,950",Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,...,Custom Cars West,"[('6/01/22', 'Listed', '$40,523'), ('7/14/22',...",At least 1 accident or damage reported,No,No,At least 1 open recall reported,NaN,2019,RAM,1500 Laramie
4,https://www.cars.com/vehicledetail/36efbef5-f2...,2015 Jeep Wrangler Unlimited Rubicon,"135,385 mi.","$26,950",This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,...,Custom Cars West,"[('1/21/22', 'Listed', '$29,950'), ('3/22/22',...",None reported,No,Yes,At least 1 open recall reported,NaN,2015,Jeep,Wrangler Unlimited Rubicon


In [17]:
#df = df[['year', 'make', 'model', 'listing_mileage', 'primary_price', 'deal_gauge', 'exterior_color', 'interior_color', 'drivetrain', 'mpg', 'fuel_type', 'transmission', 'engine', 'vin', 'stock_number', 'accident', 'one_owner', 'personal_use', 'open_recall', 'seller_name', 'price_history', 'link']]

## Split price_history to listed_date, price_change, price_change_percentage

In [5]:
# Convert string representation of lists to actual lists
df['price_history'] = df['price_history'].apply(ast.literal_eval)

# Extract the oldest date, total price change, and price change percentage
df['listing_date'] = df['price_history'].apply(lambda x: min([pd.to_datetime(item[0], format='%m/%d/%y')for item in x], default=None))

# Calculate total price change and price change percentage
def calculate_changes(prices):
    if not prices:
        return None, None
    prices = [(pd.to_datetime(date, format='%m/%d/%y'), status, int(price.replace('$', '').replace(',', ''))) for date, status, price in prices]
    total_price_change = prices[-1][2] - prices[0][2]
    price_change_percentage = (total_price_change / prices[0][2]) * 100
    return total_price_change, price_change_percentage

df['total_price_change'], df['price_change_percentage'] = zip(*df['price_history'].apply(calculate_changes))
df

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,...,1-owner vehicle,Personal use only,Open recall,Clean title,year,make,model,listing_date,total_price_change,price_change_percentage
0,https://www.cars.com/vehicledetail/5ee3ae04-69...,2018 Hyundai Santa Fe Sport 2.4L,"79,497 mi.","$17,899",Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,...,Yes,No,At least 1 open recall reported,NaN,2018,Hyundai,Santa Fe Sport 2.4L,2021-09-21,-2400.0,-11.823243
1,https://www.cars.com/vehicledetail/c99e5035-76...,2017 RAM 1500 Express,"80,326 mi.","$27,900",This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,...,No,Yes,NaN,NaN,2017,RAM,1500 Express,2022-03-15,-3085.0,-9.956431
2,https://www.cars.com/vehicledetail/4e79fa3a-ba...,2019 Nissan Kicks SR,"92,959 mi.","$18,950",This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,...,No,Yes,NaN,NaN,2019,Nissan,Kicks SR,2022-04-16,-1030.0,-5.155155
3,https://www.cars.com/vehicledetail/86a0d096-32...,2019 RAM 1500 Laramie,"59,176 mi.","$39,950",Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,...,No,No,At least 1 open recall reported,NaN,2019,RAM,1500 Laramie,2022-06-01,-573.0,-1.414012
4,https://www.cars.com/vehicledetail/36efbef5-f2...,2015 Jeep Wrangler Unlimited Rubicon,"135,385 mi.","$26,950",This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,...,No,Yes,At least 1 open recall reported,NaN,2015,Jeep,Wrangler Unlimited Rubicon,2022-01-21,-3000.0,-10.016694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,https://www.cars.com/vehicledetail/58470668-1c...,2022 Toyota Tacoma TRD Off Road,"32,443 mi.","$36,749","Great Deal $2,494 under",-,-,Four-wheel Drive,18-22,Gasoline,...,Yes,No,NaN,Yes,2022,Toyota,Tacoma TRD Off Road,NaT,NaN,NaN
5998,https://www.cars.com/vehicledetail/0d3c6ec5-e0...,2022 Nissan Kicks SV,"35,000 mi.","$18,834",Good Deal $74 under,Fresh Powder,Charcoal,Front-wheel Drive,31-36,Gasoline,...,Yes,No,NaN,Yes,2022,Nissan,Kicks SV,2023-10-28,-490.0,-2.535707
5999,https://www.cars.com/vehicledetail/7b40bf89-08...,2022 Ford Expedition XLT,"23,477 mi.","$50,599",This is a good deal.,Oxford White,Black,Four-wheel Drive,16-22,Gasoline,...,Yes,No,NaN,Yes,2022,Ford,Expedition XLT,NaT,NaN,NaN
6000,https://www.cars.com/vehicledetail/0fc31017-d3...,2022 Nissan Altima 2.5 S,"47,518 mi.","$18,199",Good Deal $271 under,Brilliant Silver Metallic,Charcoal,Front-wheel Drive,28-39,Gasoline,...,Yes,No,NaN,Yes,2022,Nissan,Altima 2.5 S,NaT,NaN,NaN


In [6]:
df.shape

(6002, 28)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6002 entries, 0 to 6001
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   link                     6002 non-null   object        
 1   listing_title            6002 non-null   object        
 2   listing_mileage          6002 non-null   object        
 3   primary_price            6002 non-null   object        
 4   deal_gauge               5524 non-null   object        
 5   exterior_color           6002 non-null   object        
 6   interior_color           6002 non-null   object        
 7   drivetrain               6002 non-null   object        
 8   mpg                      5957 non-null   object        
 9   fuel_type                6002 non-null   object        
 10  transmission             6002 non-null   object        
 11  engine                   6002 non-null   object        
 12  vin                      6002 non-

In [13]:
# Check unique values to see what we're working with: 
# engine, transmission, drivetrain. 
# might need to standardize for the model to use.

unique_engines = sorted(df['engine'].unique())
unique_transmissions = sorted(df['transmission'].unique())
unique_drivetrains = sorted(df['drivetrain'].unique())
unique_fuel_types = sorted(df['fuel_type'].unique())

# Print the unique values
print("Sorted Unique Engines:")
for engine in unique_engines:
    print(engine)
# Want to isolate: 
# cylinders
# liters
# "Turbo" T/F
# "Supercharged" T/F

print("\nSorted Unique Transmissions:")
for transmission in unique_transmissions:
    print(transmission)
# Not worth the effort to differentiate automatic versus manual

print("\nSorted Unique Drivetrains:")
for drivetrain in unique_drivetrains:
    print(drivetrain)
# Doesn't need to be standardized, only a few values

# Verify that fuel_type is always Gasoline
print("\nSorted Unique Fuel Types:")
for fuel_type in unique_fuel_types:
    print(fuel_type)
# Only Gasoline, column can be removed.

Sorted Unique Engines:
-
1.0L I3 12V GDI DOHC Turbo
1.3L I3 12V GDI DOHC Turbo
1.3L I3 12V MPFI DOHC Turbo
1.3L I4 16V GDI SOHC Turbo
1.4L I-4 DOHC, variable valve control, regular unleaded, engine
1.4L I-4 gasoline direct injection, DOHC, variable valve control
1.4L I4
1.4L I4 16V GDI DOHC Turbo
1.4L I4 16V MPFI DOHC
1.4L I4 16V MPFI DOHC Turbo
1.4L I4 16V MPFI SOHC
1.4L I4 16V MPFI SOHC Turbo
1.4L TSI
1.5L EcoBoost
1.5L I-4 DOHC, variable valve control, regular unleaded, engine
1.5L I-4 gasoline direct injection, DOHC, VTEC variable valve co
1.5L I-4 gasoline direct injection, DOHC, VVT variable valve con
1.5L I-4 gasoline direct injection, DOHC, i-VTEC variable valve
1.5L I-4 gasoline direct injection, DOHC, variable valve control
1.5L I-4 variable valve control, regular unleaded, engine with 1
1.5L I3 12V GDI DOHC Turbo
1.5L I3 12V PDI DOHC Turbo
1.5L I4 16V GDI DOHC
1.5L I4 16V GDI DOHC Turbo
1.5L I4 16V MPFI SOHC
1.5L I4 DOHC 16V
1.5L I4 Turbocharged DOHC 16V LEV3-SULEV30 158hp
1

## Replace - in exterior color & interior color to NaN

In [ ]:
df['exterior_color'] = df['exterior_color'].replace('-', np.nan)
df['interior_color'] = df['interior_color'].replace('-', np.nan)

In [92]:
df.to_csv('new_cars.csv',index=False)